In [1]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from tqdm import tqdm
import sys
sys.path.append("..")

import config

In [2]:
domain = "10YBE----------2"

types = pd.read_csv("../PsrType.csv", sep=";")
types.set_index("Code", inplace=True)

dates = range(2015, 2023)
generationCAP_df = pd.DataFrame([], columns = ["datetime", "type", "capacity"])

In [3]:
def get_response(year, BZ):
    
    """
    This function takes in a year and a bidding zone.
    It calls the API, and returns the API response.
    This can be fed to the data format function.
    """
    
    startdate = str(year) + "01010000"
    enddate = str(year) + "12312300"
    domain = BZ
    
    try:
        response = requests.get("https://web-api.tp.entsoe.eu/api",
                        params = {"securityToken": config.security_token,
                                "documentType": "A68",
                                "processType": "A33",
                                "in_Domain": domain,
                                "periodStart": startdate,
                                "periodEnd": enddate})
    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
        
    return response

In [4]:
def parse_data(resp):
    """
    This function takes in a response from the API.
    It returns a pandas dataframe with variables start and price. 
    Price is a number, and start is a datetime variable.
    The index starts at 0 and goes until the end.
    This function is to be executed per year, per bidding zone.
    Resulting dataframes can then later be combined.
    """
    response = resp

    root = ET.fromstring(response.content)
    timeseries = root.findall("./{urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0}TimeSeries")

    df_cols = ["datetime", "type", "capacity"]
    df_rows = []

    for ts in timeseries:
        data = ts[7]
        df_rows.append({
            "type": ts[6][0].text,
            "datetime": year,
            "capacity": data[2][1].text
        })

    out_df = pd.DataFrame(df_rows, columns = df_cols)
    out_df["type"] = types.loc[out_df["type"], "Meaning"].values

    #print(f"Resolution {data[1].text}")

    return out_df

In [5]:
for year in tqdm(dates):
    resp = get_response(year, domain)
    out_df = parse_data(resp)
    generationCAP_df = pd.concat([generationCAP_df, out_df], axis=0, ignore_index=True)

100%|█████████████████████████████████████████████| 8/8 [00:05<00:00,  1.43it/s]


In [7]:
generationCAP_df.to_csv("../data/temp/generationCAP.csv", index=False)